In [ ]:
!pip install huggingface_hub --upgrade
!pip install diffusers accelerate transformers safetensors sentencepiece
!pip install clean-fid

In [ ]:
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
import torch
from huggingface_hub import login
import numpy as np
import os
from PIL import Image
import json
import os.path
from cleanfid import fid

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
#Initialize Stable Diffusion model
model_id = "OFA-Sys/small-stable-diffusion-v0"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
with open("/content/drive/MyDrive/FinalProject/data/demo_prompts.json", "r") as file:
  prompts = json.load(file)

dir_name = "/content/drive/MyDrive/FinalProject/data/Paper-Poster_Demo/SSD_Generated/"

if not os.path.exists(dir_name):
  os.makedirs(dir_name)

images_path = '/content/drive/MyDrive/FinalProject/data/Paper-Poster_Demo/Original/'

# List of sample true images
image_names = ['37i9dQZF1DWWOaP4H0w5b0_0.jpg', '0OmxoVx5pWx8ARiI9rEJML.jpg', '0Iy20da7GRpAQGfVXZ5CFB.jpg', '216GOIIed7qbRTVVDDDES4.jpg', '37i9dQZF1DWV7cvDzE3MOI.jpg', "0NSj06X0oIoDaBkR9MxB2c.jpg"]

# Generate and save the output images as well as the prompts
for i in range(len(image_names)):
  image_id = image_names[i][0:22]
  prompt = prompts[image_id]
  print(prompt)
  generated_image = pipe(prompt).images[0]
  plt.imshow(generated_image)
  generated_image.save(dir_name + image_names[i])

In [ ]:
dir_orig = "/content/drive/MyDrive/FinalProject/data/SSDBaseline/Original/"
if not os.path.exists(dir_orig):
  os.makedirs(dir_orig)

dir_gen = "/content/drive/MyDrive/FinalProject/data/SSDBaseline/Generated/"
if not os.path.exists(dir_gen):
  os.makedirs(dir_gen)

with open("/content/drive/MyDrive/FinalProject/data/test_set_prompts.json", "r") as file:
  prompts = json.load(file)

test_path = '/content/drive/MyDrive/FinalProject/data/test/'

# Get a list of all file names in the folder
file_names = os.listdir(test_path)

# Size of the test set
num_test = len(file_names)

# Generate an image using the model for each prompt
for i in range(num_test):
  image = Image.open(test_path + file_names[i])
  image.save(dir_orig + file_names[i])
  image_id = file_names[i][0:22]
  prompt = prompts[image_id]
  generated_image = pipe(prompt).images[0]
  generated_image.save(dir_gen + file_names[i])

In [ ]:
# Calculate the FID score
dir_orig = "/content/drive/MyDrive/FinalProject/data/DFOrig/"
dir_gen = "/content/drive/MyDrive/FinalProject/data/DFGen2/"
score = fid.compute_fid(dir_orig, dir_gen)
print("FID score: " + str(score))

In [ ]:
# Calm nights generation for poster/paper demo
dir_name = "/content/drive/MyDrive/FinalProject/data/Paper-Poster_Demo/SSD_Generated/"
test_path = '/content/drive/MyDrive/FinalProject/data/test/'
image = Image.open(test_path + "0NSj06X0oIoDaBkR9MxB2c.jpg")
generated_image = pipe("Calm Nights playlist cover image").images[0]
generated_image.save(dir_name + "0NSj06X0oIoDaBkR9MxB2c.jpg")

In [ ]:
# Fine-tuning command
accelerate launch train_text_to_image.py --pretrained_model_name_or_path="OFA-Sys/small-stable-diffusion-v0" --train_data_dir="images" --use_ema --resolution=256 --center_crop --random_flip --train_batch_size=1 --gradient_accumulation_steps=1 --gradient_checkpointing --checkpointing_steps=5000 --mixed_precision="fp16" --max_train_steps=70000 --learning_rate=1e-10 --max_grad_norm=1 --lr_scheduler="constant" --lr_warmup_steps=0 --output_dir="output" 